In [ ]:
#@markdown **Make sure that your colab session is connected to a GPU.**
!nvidia-smi

# **Installation**

In [ ]:
#@title Step 1
%%bash
mkdir /content/env
MINICONDA_INSTALLER_SCRIPT=Miniconda3-py310_24.5.0-0-Linux-x86_64.sh
MINICONDA_PREFIX=/content/env
wget -q --show-progress https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
#@title Step 2
%%bash
source /content/env/bin/activate
conda create -n fish_diffusion python=3.10 -y

In [ ]:
#@title Step 3
import os

if os.path.exists("/content/fish-diffusion"):
  print("The repo alerady exists, skipping")
else:
  !wget https://github.com/usamireko/DiffusionDenoiser/releases/download/download/fish-diffusion.zip
  !unzip /content/fish-diffusion.zip -d /content
  !wget https://github.com/fishaudio/fish-diffusion/releases/download/v2.2.1/denoiser-cn-hubert-large-v1.ckpt -O /content/fish-diffusion/denoiser-cn-hubert-large-v1.ckpt

In [ ]:
#@title Step 4 (This one can take a bit of time)
%%capture
%cd /content/fish-diffusion
!source /content/env/bin/activate;\
conda activate fish_diffusion;\
curl -sSL https://raw.githubusercontent.com/pdm-project/pdm/main/install-pdm.py | python3 -;\
/root/.local/bin/pdm sync;

In [ ]:
#@markdown Step 5
from IPython.display import display, HTML, clear_output
%cd /content/fish-diffusion
!source /content/env/bin/activate;\
conda activate fish_diffusion;\
python tools/download_nsf_hifigan.py --agree-license

display(HTML(
      '''
      Installation successfully!!



      '''))


# **Explaining parameters**

*First time will take time to start, after the 1st run it´ll be faster*

*  Input Audio: Path where the audio is located
*  Output Audio: Path where the processed audio will be dropped
*  N Steps: Amount of steps done by the diffusion denoiser, so the default will only make 30 steps. Less steps can help preserve accent!
*  Sampler Interval: Smaller number could lead to higher attention to detail but slower inference times and vice-verse




```
In case of N Steps and Sampler Interval only touch them if you know what are you doing or if you want to experiment!
```






In [ ]:
Input_Audio = "" # @param {type:"string"}
Output_Audio = "" # @param {type:"string"}
N_Steps = 30  # @param {type:"integer", min:1, max:999}
Skip_Steps = 1000 - N_Steps
Sampler_Interval = 10 # @param {type:"integer"}
# %cd /content/fish-diffusion
!source /content/env/bin/activate;\
    conda activate fish_diffusion;\
    python tools/diffusion/inference.py --config configs/denoiser_cn_hubert.py \
    --checkpoint denoiser-cn-hubert-large-v1.ckpt \
    --input {Input_Audio} \
    --output {Output_Audio} \
    --sampler_interval {Sampler_Interval} \
    --skip_steps {Skip_Steps}

In [ ]:
#@title Play the result
wav_name = "" #@param {type:"string"}
rate=44100
from IPython.display import Audio, display

display(Audio(wav_name, rate=rate, autoplay=True))